In [1]:
using POMDPs
using Random # for AbstractRNG
using POMDPModelTools
using Pkg
Pkg.add("JSON")

POMDPs.add_registry()
using Pkg; Pkg.add("DiscreteValueIteration")

using DiscreteValueIteration

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Updating registry at `~/.julia/registries/JuliaPOMDP`
  Updating git-repo `https://github.com/JuliaPOMDP/Registry`
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
   Cloning registry from "https://github.com/JuliaPOMDP/Registry"
Fetching: [========================================>]  100.0 %.0 %=========>                               ]  21.4 %]  42.7 %==========================>              ]  64.1 %]  85.3 %

┌ Info: registry `JuliaPOMDP` already exists in `~/.julia/registries`
└ @ POMDPs /Users/efan/.julia/packages/POMDPs/JiYXY/src/utils.jl:65


 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


In [ ]:
using Pkg
Pkg.add("StaticArrays")

In [ ]:

include("./ParallelChains.jl")

In [ ]:
using POMDPSimulators
using POMDPPolicies
rng = MersenneTwister(1235);
num_chains = 5
num_states = 6
r_mean = 5.0
r_std = 2.0
PParallelChainMDP() = PFParallelChainMDP.PParallelChainMDP(num_states+1,num_chains, .9,

        randn(rng, Float32, num_chains) .* r_std .+ r_mean)
m = ParallelChainMDP()

# policy that maps every input to an action
policy = FunctionPolicy(s->3)

for (s, a, r) in stepthrough(m, policy, "s,a,r", max_steps=10)
    render(m, (s,a,r))
    println("s,a,r:($s,$a,$r)")
end

In [ ]:
curry(f, a) = (xs...) -> f(a, xs...)

In [19]:
using POMDPSimulators
using POMDPPolicies
rng = MersenneTwister(1235);
include("./ParallelChains.jl")

num_chains = 5
num_states = 6
r_mean = 5.0
r_std = 2.0
@requirements_info ValueIterationSolver() PParallelChainMDP(num_states+1,num_chains, .9,

        randn(rng, Float32, num_chains) .* r_std .+ r_mean)

states:SArray{Tuple{2},Int64,1,2}[[1, 1], [2, 1], [3, 1], [4, 1], [5, 1], [1, 2], [2, 2], [3, 2], [4, 2], [5, 2], [1, 3], [2, 3], [3, 3], [4, 3], [5, 3], [1, 4], [2, 4], [3, 4], [4, 4], [5, 4], [1, 5], [2, 5], [3, 5], [4, 5], [5, 5], [1, 6], [2, 6], [3, 6], [4, 6], [5, 6], [1, 7], [2, 7], [3, 7], [4, 7], [5, 7]]
states:SArray{Tuple{2},Int64,1,2}[[1, 1], [2, 1], [3, 1], [4, 1], [5, 1], [1, 2], [2, 2], [3, 2], [4, 2], [5, 2], [1, 3], [2, 3], [3, 3], [4, 3], [5, 3], [1, 4], [2, 4], [3, 4], [4, 4], [5, 4], [1, 5], [2, 5], [3, 5], [4, 5], [5, 5], [1, 6], [2, 6], [3, 6], [4, 6], [5, 6], [1, 7], [2, 7], [3, 7], [4, 7], [5, 7]]
INFO: POMDPs.jl requirements for solve(::ValueIterationSolver, ::Union{MDP,POMDP}) and dependencies. ([✔] = implemented correctly; [X] = missing)

For solve(::ValueIterationSolver, ::Union{MDP,POMDP}):
  [✔] discount(::PParallelChainMDP)
  [✔] n_states(::PParallelChainMDP)
  [✔] n_actions(::PParallelChainMDP)
  [✔] transition(::PParallelChainMDP, ::SArray, ::Int64)
  [✔

true

In [23]:
mdp = PParallelChainMDP(num_states+1,num_chains, .9,
        randn(rng, Float32, num_chains) .* r_std .+ r_mean)
solver = ValueIterationSolver(max_iterations=100, belres=1e-6) # initializes the Solver type
vip = solve(solver, mdp) # runs value iterations

states:SArray{Tuple{2},Int64,1,2}[[1, 1], [2, 1], [3, 1], [4, 1], [5, 1], [1, 2], [2, 2], [3, 2], [4, 2], [5, 2], [1, 3], [2, 3], [3, 3], [4, 3], [5, 3], [1, 4], [2, 4], [3, 4], [4, 4], [5, 4], [1, 5], [2, 5], [3, 5], [4, 5], [5, 5], [1, 6], [2, 6], [3, 6], [4, 6], [5, 6], [1, 7], [2, 7], [3, 7], [4, 7], [5, 7]]
states:SArray{Tuple{2},Int64,1,2}[[1, 1], [2, 1], [3, 1], [4, 1], [5, 1], [1, 2], [2, 2], [3, 2], [4, 2], [5, 2], [1, 3], [2, 3], [3, 3], [4, 3], [5, 3], [1, 4], [2, 4], [3, 4], [4, 4], [5, 4], [1, 5], [2, 5], [3, 5], [4, 5], [5, 5], [1, 6], [2, 6], [3, 6], [4, 6], [5, 6], [1, 7], [2, 7], [3, 7], [4, 7], [5, 7]]
states:SArray{Tuple{2},Int64,1,2}[[1, 1], [2, 1], [3, 1], [4, 1], [5, 1], [1, 2], [2, 2], [3, 2], [4, 2], [5, 2], [1, 3], [2, 3], [3, 3], [4, 3], [5, 3], [1, 4], [2, 4], [3, 4], [4, 4], [5, 4], [1, 5], [2, 5], [3, 5], [4, 5], [5, 5], [1, 6], [2, 6], [3, 6], [4, 6], [5, 6], [1, 7], [2, 7], [3, 7], [4, 7], [5, 7]]
s:[1, 1]
li:1
s:[2, 1]
li:2
s:[3, 1]
li:3
s:[4, 1]
li:4
s:

ValueIterationPolicy{Float64}([1.78483 1.57811 … 4.51483 2.42094; 1.78483 1.57811 … 4.51483 2.42094; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [4.51483, 4.51483, 4.51483, 4.51483, 4.51483, 1.98315, 1.75346, 1.89565, 5.01648, 2.68993  …  3.02263, 2.67255, 2.88927, 7.64591, 4.09988, 0.0, 0.0, 0.0, 0.0, 0.0], [4, 4, 4, 4, 4, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 2, 3, 4, 5], true, PParallelChainMDP(7, 5, 0.9, [3.02263, 2.67255, 2.88927, 7.64591, 4.09988]))

In [21]:
mdp.Rs

5-element Array{Float64,1}:
 6.552718877792358
 4.796569257974625
 5.359664976596832
 6.065513610839844
 4.617465168237686

In [25]:
using POMDPSimulators
using POMDPPolicies



for (s, a, r) in stepthrough(mdp, vip, "s,a,r", max_steps=10)
    render(mdp, (s,a,r))
    println("s,a,r:($s,$a,$r)")
end



s:[1, 1]
li:1
s,a,r:([1, 1],4,0)
s:[4, 2]
li:9
s,a,r:([4, 2],1,0)
s:[4, 3]
li:14
s,a,r:([4, 3],1,0)
s:[4, 4]
li:19
s,a,r:([4, 4],1,0)
s:[4, 5]
li:24
s,a,r:([4, 5],1,0)
s:[4, 6]
li:29
s,a,r:([4, 6],1,7.645909547805786)
